# TRANSFORMACIÓN DE DATOS

## 1. IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

#Mostrar el máximo de columnas posibles de una tabla
pd.set_option('display.max_columns', None) #Número de columnas que deben verse. None = Máx


#Mostrar el máximo de filas posibles de una tabla
pd.set_option('display.max_rows', 100) #Número de filas que deben verse. None = Máx

## 2. IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
#Abrir el excel de las transformaciones que le vamos a hacer a los datos para hacer un esquema de aplicacion y no perdernos
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = True)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num).reset_index(drop = True)

In [5]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    18139 non-null  object
 1   host_response_time      18139 non-null  object
 2   host_is_superhost       18139 non-null  object
 3   host_verifications      18139 non-null  object
 4   host_has_profile_pic    18139 non-null  object
 5   host_identity_verified  18139 non-null  object
 6   property_type           18139 non-null  object
 7   room_type               18139 non-null  object
 8   amenities               18139 non-null  object
 9   has_availability        18139 non-null  object
 10  license                 18139 non-null  object
 11  instant_bookable        18139 non-null  object
 12  neighbourhood_group     18139 non-null  object
 13  host_since              18139 non-null  object
 14  first_review            18139 non-null  object
 15  la

4.- Separar la target. En este caso hemos separado la target previamente, por lo que la cargamos desde la base de datos y trabajamos con ella

In [6]:
target = num[['pisos_rentables']].copy().reset_index(drop=True)
target

,pisos_rentables
0,1
1,1
2,1
3,0
4,1
...,...
18134,0
18135,0
18136,0
18137,0


## 3. TRANSFORMACIÓN DE CATEGÓRICAS

### 3.1. One Hot Encoding

#### 3.1.1. Variables a aplicar OHE

In [7]:
var_ohe = ['property_type',
           'room_type',
           'neighbourhood_group',
           'bedrooms_disc',
           'accommodates_disc',
           'beds_disc']

#### 3.1.2. Instanciar

In [8]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### 3.1.3. Entrenar y aplicar

In [9]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### 3.1.4. Guardar como dataframe

In [10]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### 3.2. Ordinal Encoding

#### 3.2.1. Variables a aplicar OE

In [11]:
var_oe = ['host_response_time',
          'host_is_superhost',
          'host_verifications',
          'host_has_profile_pic',
          'host_identity_verified',
          'has_availability',
          'license',
          'instant_bookable',
          'number_of_reviews_disc']

#### 3.2.2. Orden de los valores de las variables

In [12]:
cat.number_of_reviews_disc.unique()

array(['5-71', '1-4', '71-1136'], dtype=object)

In [13]:
#Orden de la primera variable
orden_host_response_time = ['no response', 'a few days or more','within a day', 'within a few hours', 'within an hour' ]

#Orden de la segunda variable
orden_host_is_superhost = ['f','t']

#Orden de la variable
orden_host_verifications = ["['No']", "['email']", "['phone']", "['phone', 'work_email']", "['email', 'phone']", "['email', 'phone', 'work_email']"]

#Orden de la variable
orden_host_has_profile_pic = ['f','t']

#Orden de la variable
orden_host_identity_verified = ['f','t']

#Orden de la variable
orden_has_availability = ['yes','no']

#Orden de la variable
orden_license = ['no','yes']

#Orden de la variable
orden_instant_bookable = ['f','t']

#Orden de la variable
orden_number_of_reviews_disc = ['1-4', '5-71', '71-1136']

#### 3.2.3. Instanciar

In [14]:
oe = OrdinalEncoder(categories = [orden_host_response_time,
                                  orden_host_is_superhost,
                                  orden_host_verifications,
                                  orden_host_has_profile_pic,
                                  orden_host_identity_verified,
                                  orden_has_availability,
                                  orden_license,
                                  orden_instant_bookable,
                                  orden_number_of_reviews_disc
                                 ],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 10)

#### 3.2.4. Entrenar y aplicar

In [15]:
cat_oe = oe.fit_transform(cat[var_oe])

#### 3.2.5. Guardar como dataframe

In [16]:
#Añadir sufijos a los nombres
nombres_oe = [variable + '_oe' for variable in var_oe]

#Guardar como dataframe
cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)
cat_oe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_response_time_oe      18139 non-null  float64
 1   host_is_superhost_oe       18139 non-null  float64
 2   host_verifications_oe      18139 non-null  float64
 3   host_has_profile_pic_oe    18139 non-null  float64
 4   host_identity_verified_oe  18139 non-null  float64
 5   has_availability_oe        18139 non-null  float64
 6   license_oe                 18139 non-null  float64
 7   instant_bookable_oe        18139 non-null  float64
 8   number_of_reviews_disc_oe  18139 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [17]:
print(cat_oe.dtypes)
print(cat_oe.head())

host_response_time_oe        float64
host_is_superhost_oe         float64
host_verifications_oe        float64
host_has_profile_pic_oe      float64
host_identity_verified_oe    float64
has_availability_oe          float64
license_oe                   float64
instant_bookable_oe          float64
number_of_reviews_disc_oe    float64
dtype: object
   host_response_time_oe  host_is_superhost_oe  host_verifications_oe  \
0                    0.0                   0.0                    4.0   
1                    0.0                   0.0                    4.0   
2                    0.0                   0.0                    4.0   
3                    4.0                   0.0                    4.0   
4                    0.0                   0.0                    4.0   

   host_has_profile_pic_oe  host_identity_verified_oe  has_availability_oe  \
0                      1.0                        1.0                  0.0   
1                      1.0                        0.0     

### 3.4. Fechas y textos

In [18]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    18139 non-null  object
 1   host_response_time      18139 non-null  object
 2   host_is_superhost       18139 non-null  object
 3   host_verifications      18139 non-null  object
 4   host_has_profile_pic    18139 non-null  object
 5   host_identity_verified  18139 non-null  object
 6   property_type           18139 non-null  object
 7   room_type               18139 non-null  object
 8   amenities               18139 non-null  object
 9   has_availability        18139 non-null  object
 10  license                 18139 non-null  object
 11  instant_bookable        18139 non-null  object
 12  neighbourhood_group     18139 non-null  object
 13  host_since              18139 non-null  object
 14  first_review            18139 non-null  object
 15  la

In [19]:
def modificaciones_fechas(cat):
    
    def convertir_fecha(fecha):
        for formato in ['%Y-%m-%d', '%d/%m/%Y', '%d-%m-%Y', '%Y-%d-%m', '%m/%d/%Y']:
            try:
                return pd.to_datetime(fecha, format=formato)
            except ValueError:
                continue
        return pd.NaT  # Si no coincide con ningún formato

    # Aplicar la función a la columna, asegurando que el DataFrame se mantenga
    cat['host_since'] = cat['host_since'].apply(convertir_fecha)
    cat['first_review'] = cat['first_review'].apply(convertir_fecha)
    cat['last_review'] = cat['last_review'].apply(convertir_fecha)

    # Convertir las fechas al formato deseado y sobrescribir la columna original
    cat['host_since'] = pd.to_datetime(cat['host_since'], dayfirst=True, errors='coerce')
    cat['first_review'] = pd.to_datetime(cat['first_review'], dayfirst=True, errors='coerce')
    cat['last_review'] = pd.to_datetime(cat['last_review'], dayfirst=True, errors='coerce')

    def componentes_fecha(cat):

        cat.reset_index(inplace=True)

        #host_since
        cat.set_index('host_since', inplace=True)    
        cat['host_since_year'] = cat.index.year

        #first_review
        cat.set_index('first_review', inplace=True)    
        cat['first_review_year'] = cat.index.year

        #last_review
        cat.set_index('last_review', inplace=True)    
        cat['last_review_year'] = cat.index.year

        #cat.reset_index(inplace=True)

        return(cat)
    cat = componentes_fecha(cat)
    cat['host_since_year'] = cat['host_since_year'].astype('O')
    cat['first_review_year'] = cat['first_review_year'].astype('O')
    cat['last_review_year'] = cat['last_review_year'].astype('O')
    return(cat)

cat = modificaciones_fechas(cat)

In [20]:
def variables_texto(cat):    
    #Cargamos la lista de stopwords en un dataframe. La lista la tendremos que descargar y ubicar antes de hacer nada 
    stopwords = pd.read_csv('C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/01_Documentos/stop_words_spanish.txt',names = ['termino'])

    #Función para quitar todas las tides de las vocales
    def quitar_tildes(palabra):
        #Definimos la versión con tildes y símbolos y la sin
        con = 'áéíóúüñÁÉÍÓÚÜÑ'
        sin = 'aeiouunAEIOUUN'
        #Creamos un traductor
        traductor = str.maketrans(con,sin)
        #Aplicamos el traductor y devolvemos la palabra limpia
        return(palabra.translate(traductor))

    #Creamos una nueva columna en el dataframe de stopwords llamada 'limpias'
    stopwords['limpias'] = stopwords.termino.transform(quitar_tildes)

    #Cargamos el modelo
    from sklearn.feature_extraction.text import CountVectorizer

    #Instanciamos
    cv = CountVectorizer(strip_accents = 'unicode',
                         stop_words = stopwords.limpias.to_list(),
                         max_df = 0.7,
                         min_df = 50,
                         max_features = 50,
                         ngram_range = (1,3),#Contemplará como términos correctos los que estén compuestos por 1, 2 y 3 palabras
                         binary=True)

    #Entrenamos
    cv.fit(cat.amenities)

    #Aplicamos
    caracteristicas = cv.transform(cat.amenities)
    caracteristicas

    #Podemos ver el vocabulario total encontrado y su número de columna.
    tt = cv.vocabulary_
    tt

    #Para pasar a dataframe el vocabulario encontrado con sus nombres de columnas y ordenado
    vo = pd.DataFrame(list(tt.items()), columns=['Termino', 'Indice']).sort_values('Indice')
    vo

    #Definimos un dataframe con los términos encontrados transformados a variables que mapean las columnas donde se encuentran
    terminos = pd.DataFrame(caracteristicas.toarray(), columns=cv.get_feature_names_out())
    return(terminos)
df_terminos = variables_texto(cat)
df_terminos



,air,air conditioning,allowed,and silverware,basics,bed,bed linens,body,body soap,cleaning,clothing,clothing storage,coffee,coffee maker,conditioning,cooking,cooking basics,dedicated,dedicated workspace,dishes,dishes and,dishes and silverware,dryer bed,dryer bed linens,elevator,freezer,hair,hair dryer,hair dryer bed,hot water iron,in,iron,kitchen essentials,linens,maker,microwave,microwave hangers,oven,parking,refrigerator,room,shampoo,silverware,soap,storage,stove,tv washer,water iron,wifi kitchen,workspace
0,1,1,0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,1,0,1
4,0,0,0,1,1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18134,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1
18135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18136,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1
18137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


## 4. TRANSFORMACIÓN DE NUMÉRICAS

In [21]:
list(num)

['index',
 'host_id',
 'host_response_rate',
 'host_acceptance_rate',
 'host_listings_count',
 'host_total_listings_count',
 'latitude',
 'longitude',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'price',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'reviews_per_month',
 'precio_m2',
 'm2',
 'pdi_sol',
 'pisos_rentables']

### 4.1. Discretizar variables

#### 4.1.1. Mediante distancia similar

##### Variables a discretizar mediante distancia similar

In [22]:
var_disc_ds = ['bathrooms',
 'price',
 'minimum_nights',
 'maximum_nights',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

##### Instanciar

In [23]:
disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

##### Entrenar y aplicar

In [24]:
num_disc_ds = disc_ds.fit_transform(num[var_disc_ds])

C:\Users\Oscar\miniconda3\envs\007_airbnb\Lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


##### Guardar como dataframe

In [25]:
#Añadir sufijos a los nombres
nombres_ds = [variable + '_disc_ds' for variable in var_disc_ds]

#Guardar como dataframe
num_disc_ds = pd.DataFrame(num_disc_ds,columns = nombres_ds)

### Normalizar (Gauss)

#### Con Yeo-Johnson

##### Variables a normalizar con Yeo-Johnson

In [26]:
var_yeo = ['host_response_rate',
           'accommodates',
           'maximum_nights',
           'price',
           'availability_30',
           'availability_60',
           'availability_90',
           'availability_365']

##### Instanciar

In [27]:
yeo = PowerTransformer(method = 'yeo-johnson')

##### Entrenar y aplicar

In [28]:
num_yeo = yeo.fit_transform(num[var_yeo])

##### Guardar como dataframe

In [29]:
#Añadir sufijos a los nombres
nombres_yeo = [variable + '_yeo' for variable in var_yeo]

#Guardar como dataframe
num_yeo = pd.DataFrame(num_yeo,columns = nombres_yeo)

## 5. UNIFICAR DATASETS TRANSFORMADOS

### 5.1. Meter en una lista todos los dataframes generados

In [30]:
dataframes = []
dataframes.extend(value for name, value in locals().items() if name.startswith('cat_') or name.startswith('num_'))

### 5.2. Unir todos los dataframes

In [31]:
df = pd.concat(dataframes, axis = 1)

In [32]:
df

,property_type_Entire condo,property_type_Entire loft,property_type_Entire rental unit,property_type_OTROS,property_type_Private room in home,property_type_Private room in rental unit,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_group_Arganzuela,neighbourhood_group_Barajas,neighbourhood_group_Carabanchel,neighbourhood_group_Centro,neighbourhood_group_Chamartín,neighbourhood_group_Chamberí,neighbourhood_group_Ciudad Lineal,neighbourhood_group_Fuencarral,neighbourhood_group_Hortaleza,neighbourhood_group_Latina,neighbourhood_group_Moncloa,neighbourhood_group_Moratalaz,neighbourhood_group_Puente de Vallecas,neighbourhood_group_Retiro,neighbourhood_group_Salamanca,neighbourhood_group_San Blas,neighbourhood_group_Tetuán,neighbourhood_group_Usera,neighbourhood_group_Vicálvaro,neighbourhood_group_Villa de Vallecas,neighbourhood_group_Villaverde,bedrooms_disc_00_Estudio,bedrooms_disc_01_Una,bedrooms_disc_02_Dos,bedrooms_disc_03_Tres,bedrooms_disc_04_Cuatro o mas,accommodates_disc_0-3,accommodates_disc_4,accommodates_disc_4-16,beds_disc_1,beds_disc_3,beds_disc_3-19,host_response_time_oe,host_is_superhost_oe,host_verifications_oe,host_has_profile_pic_oe,host_identity_verified_oe,has_availability_oe,license_oe,instant_bookable_oe,number_of_reviews_disc_oe,bathrooms_disc_ds,price_disc_ds,minimum_nights_disc_ds,maximum_nights_disc_ds,calculated_host_listings_count_disc_ds,calculated_host_listings_count_entire_homes_disc_ds,calculated_host_listings_count_private_rooms_disc_ds,calculated_host_listings_count_shared_rooms_disc_ds,host_response_rate_yeo,accommodates_yeo,maximum_nights_yeo,price_yeo,availability_30_yeo,availability_60_yeo,availability_90_yeo,availability_365_yeo
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.841214,-1.644854,-1.384213,-1.923469,-1.276467,-1.394559,-1.451823,0.234681
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.841214,-0.535339,0.650876,-0.013430,-1.276467,-1.394559,-1.451823,-1.568415
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.841214,0.179739,-0.644550,0.449509,-1.276467,-1.394559,-1.451823,-1.568415
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.616480,0.696515,-1.477667,-0.483995,0.642302,0.715253,0.662194,1.043166
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.841214,-0.535339,-0.088396,0.086677,-1.276467,-1.394559,-1.451823,-1.568415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18134,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.616480,-0.535339,-0.088396,1.482568,1.452667,1.271103,1.163169,1.152818
18135,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

## 6.  REESCALAR VARIABLES

### 6.1. Con Min-Max

In [33]:
#Sacamos un info porque lo ideal es reescalar las variables que no hayan sido resultado de un OneHoteEncoding
list(df)

['property_type_Entire condo',
 'property_type_Entire loft',
 'property_type_Entire rental unit',
 'property_type_OTROS',
 'property_type_Private room in home',
 'property_type_Private room in rental unit',
 'room_type_Entire home/apt',
 'room_type_Private room',
 'room_type_Shared room',
 'neighbourhood_group_Arganzuela',
 'neighbourhood_group_Barajas',
 'neighbourhood_group_Carabanchel',
 'neighbourhood_group_Centro',
 'neighbourhood_group_Chamartín',
 'neighbourhood_group_Chamberí',
 'neighbourhood_group_Ciudad Lineal',
 'neighbourhood_group_Fuencarral',
 'neighbourhood_group_Hortaleza',
 'neighbourhood_group_Latina',
 'neighbourhood_group_Moncloa',
 'neighbourhood_group_Moratalaz',
 'neighbourhood_group_Puente de Vallecas',
 'neighbourhood_group_Retiro',
 'neighbourhood_group_Salamanca',
 'neighbourhood_group_San Blas',
 'neighbourhood_group_Tetuán',
 'neighbourhood_group_Usera',
 'neighbourhood_group_Vicálvaro',
 'neighbourhood_group_Villa de Vallecas',
 'neighbourhood_group_Villa

In [34]:
df

,property_type_Entire condo,property_type_Entire loft,property_type_Entire rental unit,property_type_OTROS,property_type_Private room in home,property_type_Private room in rental unit,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_group_Arganzuela,neighbourhood_group_Barajas,neighbourhood_group_Carabanchel,neighbourhood_group_Centro,neighbourhood_group_Chamartín,neighbourhood_group_Chamberí,neighbourhood_group_Ciudad Lineal,neighbourhood_group_Fuencarral,neighbourhood_group_Hortaleza,neighbourhood_group_Latina,neighbourhood_group_Moncloa,neighbourhood_group_Moratalaz,neighbourhood_group_Puente de Vallecas,neighbourhood_group_Retiro,neighbourhood_group_Salamanca,neighbourhood_group_San Blas,neighbourhood_group_Tetuán,neighbourhood_group_Usera,neighbourhood_group_Vicálvaro,neighbourhood_group_Villa de Vallecas,neighbourhood_group_Villaverde,bedrooms_disc_00_Estudio,bedrooms_disc_01_Una,bedrooms_disc_02_Dos,bedrooms_disc_03_Tres,bedrooms_disc_04_Cuatro o mas,accommodates_disc_0-3,accommodates_disc_4,accommodates_disc_4-16,beds_disc_1,beds_disc_3,beds_disc_3-19,host_response_time_oe,host_is_superhost_oe,host_verifications_oe,host_has_profile_pic_oe,host_identity_verified_oe,has_availability_oe,license_oe,instant_bookable_oe,number_of_reviews_disc_oe,bathrooms_disc_ds,price_disc_ds,minimum_nights_disc_ds,maximum_nights_disc_ds,calculated_host_listings_count_disc_ds,calculated_host_listings_count_entire_homes_disc_ds,calculated_host_listings_count_private_rooms_disc_ds,calculated_host_listings_count_shared_rooms_disc_ds,host_response_rate_yeo,accommodates_yeo,maximum_nights_yeo,price_yeo,availability_30_yeo,availability_60_yeo,availability_90_yeo,availability_365_yeo
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.841214,-1.644854,-1.384213,-1.923469,-1.276467,-1.394559,-1.451823,0.234681
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.841214,-0.535339,0.650876,-0.013430,-1.276467,-1.394559,-1.451823,-1.568415
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.841214,0.179739,-0.644550,0.449509,-1.276467,-1.394559,-1.451823,-1.568415
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.616480,0.696515,-1.477667,-0.483995,0.642302,0.715253,0.662194,1.043166
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.841214,-0.535339,-0.088396,0.086677,-1.276467,-1.394559,-1.451823,-1.568415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18134,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.616480,-0.535339,-0.088396,1.482568,1.452667,1.271103,1.163169,1.152818
18135,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

#### 6.1.1. Variables a reescalar con Min-Max

In [35]:
var_mms = ['host_response_time_oe',
 'host_is_superhost_oe',
 'host_verifications_oe',
 'host_has_profile_pic_oe',
 'host_identity_verified_oe',
 'has_availability_oe',
 'license_oe',
 'instant_bookable_oe',
 'number_of_reviews_disc_oe',
 'price_disc_ds',
 'minimum_nights_disc_ds',
 'maximum_nights_disc_ds',
 'calculated_host_listings_count_disc_ds',
 'calculated_host_listings_count_entire_homes_disc_ds',
 'calculated_host_listings_count_private_rooms_disc_ds',
 'calculated_host_listings_count_shared_rooms_disc_ds',
 'host_response_rate_yeo',
 'accommodates_yeo',
 'maximum_nights_yeo',
 'price_yeo',
 'availability_30_yeo',
 'availability_60_yeo',
 'availability_90_yeo',
 'availability_365_yeo']

#### 6.1.2. Instanciar

In [36]:
mms = MinMaxScaler()

#### 6.1.3. Entrenar y aplicar

In [37]:
df_mms = mms.fit_transform(df[var_mms])

#### 6.1.4. Guardar como dataframe

In [38]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]
#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)
df_mms

,host_response_time_oe_mms,host_is_superhost_oe_mms,host_verifications_oe_mms,host_has_profile_pic_oe_mms,host_identity_verified_oe_mms,has_availability_oe_mms,license_oe_mms,instant_bookable_oe_mms,number_of_reviews_disc_oe_mms,price_disc_ds_mms,minimum_nights_disc_ds_mms,maximum_nights_disc_ds_mms,calculated_host_listings_count_disc_ds_mms,calculated_host_listings_count_entire_homes_disc_ds_mms,calculated_host_listings_count_private_rooms_disc_ds_mms,calculated_host_listings_count_shared_rooms_disc_ds_mms,host_response_rate_yeo_mms,accommodates_yeo_mms,maximum_nights_yeo_mms,price_yeo_mms,availability_30_yeo_mms,availability_60_yeo_mms,availability_90_yeo_mms,availability_365_yeo_mms
0,0.00,0.0,0.8,1.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.185619,0.106321,0.000000,0.000000,0.000000,0.662603
1,0.00,0.0,0.8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.237467,0.817539,0.339678,0.000000,0.000000,0.000000,0.000000
2,0.00,0.0,0.8,1.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.390513,0.415293,0.396237,0.000000,0.000000,0.000000,0.000000
3,1.00,0.0,0.8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.501117,0.156600,0.282187,0.703069,0.791478,0.808422,0.959705
4,0.00,0.0,0.8,1.0,1.0,0.0,1.0,0.0,0.5,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.237467,0.587986,0.351908,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18134,0.75,0.0,0.8,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,1.0,0.237467,0.587986,0.522449,1.000000,1.000000,1.000000,1.000000
18135,1.00,0.0,0.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.237467,0.100457,0.324158,1.000000,1.000000,1.000000,1.000000
18136,0.75,0.0,0.8,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,1.0,0.501117,0.587986,0.522449,0.749657,0.871158,0.912922,0.981971
18137,0.00,0.0,0.8,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.104959,0.000000,0.986780,0.992066,0.994462,0.998812


## 7. UNIFICAR DATASETS REESCALADOS

### 7.1. Crear una lista con los dataframes a incluir en el tablón analítico

In [39]:
incluir = [df, 
           df_mms, #df MinMax Scaler
           #df_rs, #df Robust Scaler
           #df_ss, #df Standard Scaler
           df_terminos,
           target
           ]

### 7.2. Unir todos los dataframes en el tablón analítico

In [40]:
df_tablon = pd.concat(incluir, axis = 1)

In [41]:
#Detectamos que se han generado 7774 nulos 
df_tablon.isna().sum().head()

property_type_Entire condo            0
property_type_Entire loft             0
property_type_Entire rental unit      0
property_type_OTROS                   0
property_type_Private room in home    0
dtype: int64

In [42]:
criterio = 3 #Mínimo de variables SIN nulos por registro. Se elimina el registro si hay menos de esas variables sin nulos
df_tablon.dropna(thresh=criterio, inplace=True)
df_tablon.isna().sum().sort_values(ascending=True)

property_type_Entire condo    0
coffee                        0
clothing storage              0
clothing                      0
cleaning                      0
                             ..
license_oe                    0
instant_bookable_oe           0
number_of_reviews_disc_oe     0
host_verifications_oe         0
pisos_rentables               0
Length: 141, dtype: int64

## 8. GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [43]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [44]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)

In [45]:
df_tablon.head(100)

,property_type_Entire condo,property_type_Entire loft,property_type_Entire rental unit,property_type_OTROS,property_type_Private room in home,property_type_Private room in rental unit,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_group_Arganzuela,neighbourhood_group_Barajas,neighbourhood_group_Carabanchel,neighbourhood_group_Centro,neighbourhood_group_Chamartín,neighbourhood_group_Chamberí,neighbourhood_group_Ciudad Lineal,neighbourhood_group_Fuencarral,neighbourhood_group_Hortaleza,neighbourhood_group_Latina,neighbourhood_group_Moncloa,neighbourhood_group_Moratalaz,neighbourhood_group_Puente de Vallecas,neighbourhood_group_Retiro,neighbourhood_group_Salamanca,neighbourhood_group_San Blas,neighbourhood_group_Tetuán,neighbourhood_group_Usera,neighbourhood_group_Vicálvaro,neighbourhood_group_Villa de Vallecas,neighbourhood_group_Villaverde,bedrooms_disc_00_Estudio,bedrooms_disc_01_Una,bedrooms_disc_02_Dos,bedrooms_disc_03_Tres,bedrooms_disc_04_Cuatro o mas,accommodates_disc_0-3,accommodates_disc_4,accommodates_disc_4-16,beds_disc_1,beds_disc_3,beds_disc_3-19,host_response_time_oe,host_is_superhost_oe,host_verifications_oe,host_has_profile_pic_oe,host_identity_verified_oe,has_availability_oe,license_oe,instant_bookable_oe,number_of_reviews_disc_oe,bathrooms_disc_ds,price_disc_ds,minimum_nights_disc_ds,maximum_nights_disc_ds,calculated_host_listings_count_disc_ds,calculated_host_listings_count_entire_homes_disc_ds,calculated_host_listings_count_private_rooms_disc_ds,calculated_host_listings_count_shared_rooms_disc_ds,host_response_rate_yeo,accommodates_yeo,maximum_nights_yeo,price_yeo,availability_30_yeo,availability_60_yeo,availability_90_yeo,availability_365_yeo,host_response_time_oe_mms,host_is_superhost_oe_mms,host_verifications_oe_mms,host_has_profile_pic_oe_mms,host_identity_verified_oe_mms,has_availability_oe_mms,license_oe_mms,instant_bookable_oe_mms,number_of_reviews_disc_oe_mms,price_disc_ds_mms,minimum_nights_disc_ds_mms,maximum_nights_disc_ds_mms,calculated_host_listings_count_disc_ds_mms,calculated_host_listings_count_entire_homes_disc_ds_mms,calculated_host_listings_count_private_rooms_disc_ds_mms,calculated_host_listings_count_shared_rooms_disc_ds_mms,host_response_rate_yeo_mms,accommodates_yeo_mms,maximum_nights_yeo_mms,price_yeo_mms,availability_30_yeo_mms,availability_60_yeo_mms,availability_90_yeo_mms,availability_365_yeo_mms,air,air conditioning,allowed,and silverware,basics,bed,bed linens,body,body soap,cleaning,clothing,clothing storage,coffee,coffee maker,conditioning,cooking,cooking basics,dedicated,dedicated workspace,dishes,dishes and,dishes and silverware,dryer bed,dryer bed linens,elevator,freezer,hair,hair dryer,hair dryer bed,hot water iron,in,iron,kitchen essentials,linens,maker,microwave,microwave hangers,oven,parking,refrigerator,room,shampoo,silverware,soap,storage,stove,tv washer,water iron,wifi kitchen,workspace,pisos_rentables
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.841214,-1.644854,-1.384213,-1.923469,-1.276467,-1.394559,-1.451823,0.234681,0.00,0.0,0.8,1.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.185619,0.106321,0.000000,0.000000,0.000000,0.662603,1,1,0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0,1
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.841214,-0.535339,0.650876,-0.013430,-1.276467,-1.394559,-1.451823,-1.568415,0.00,0.0,0.8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.000000,0.237467,0.817539,0.339678,0.000000,0.000000,0.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0